In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip3 install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=5cf8228ef51137a2028fa70bcf427648a9457a18ee3642d54c9e12ecfd62a123
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import pandas as pd
import os
import shutil
import json
import glob
import numpy as np
import cv2
import copy
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm

In [ ]:

path_json = "/content/drive/MyDrive/Datasset/dfdc_train_part_24/metadata.json"
path_csv = "/content/drive/MyDrive/Datasset/dfdc_train_part_24/metadata.csv"
print(path_csv)
print(path_json)
read_json = pd.read_json(path_json)
df = pd.DataFrame(read_json)
df_2 = pd.DataFrame(df.transpose())
df_2.to_csv(path_csv)
read_csv = pd.read_csv(path_csv)
read_csv.columns = ["URI","label","original","split"]
read_csv.to_csv(path_csv,index=False)
print(read_csv.head(5))

/content/drive/MyDrive/Datasset/dfdc_train_part_24/metadata.csv
/content/drive/MyDrive/Datasset/dfdc_train_part_24/metadata.json
              URI label original           split
0  anijjqtfth.mp4  FAKE    train  qbwwpvgyhh.mp4
1  zffofixoeh.mp4  FAKE    train  bjdtgkmvza.mp4
2  rkbynalsjc.mp4  FAKE    train  qbwwpvgyhh.mp4
3  kyknzgpgvh.mp4  FAKE    train  hyhczxthrs.mp4
4  lklxnnznwk.mp4  FAKE    train  sxjdqerkfp.mp4


In [ ]:
video_files =  glob.glob('/content/drive/MyDrive/Data/real/*.mp4')
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [300, 300, 300, 300, 300, 299, 300, 300, 300, 300, 300, 300, 300, 300, 300, 302, 300, 300, 302, 300, 240, 300, 300, 301, 300, 300, 240, 300, 300, 300, 300, 300, 300, 300, 300, 299, 300, 288, 300, 300, 300, 300, 300, 300, 300, 300, 302, 300, 300, 300, 300, 249, 300, 288, 249, 302, 300, 301, 301, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 299, 300, 300, 300, 300, 300, 249, 300, 300, 300, 299, 240, 300, 300, 300, 300, 299, 300, 302, 300, 300, 300, 300, 300, 302, 300, 300, 300, 302, 300, 302, 300, 249, 302, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 249, 300, 300, 240, 240, 300, 249, 302, 249, 300, 249, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 302, 300, 302, 300, 300, 300, 302, 300, 300, 302, 300, 302, 300, 302, 300, 300, 300, 300, 300, 300, 300, 300, 302, 302, 300, 300, 300, 249, 300, 300, 300, 300, 302, 300, 240, 300, 300, 300, 300, 302, 300, 300, 302, 300, 302, 300, 300, 300, 240, 299, 302, 300, 300, 302, 300, 300, 302, 30

In [ ]:
# extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path)
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image


# process the frames
def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    frames = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      if(idx <= 100):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

In [ ]:
video_files =  glob.glob('/content/drive/MyDrive/Data/real/*.mp4')
create_face_videos(video_files,'/content/drive/MyDrive/Data/REAL')